In [55]:
import itertools
import operator
from operator import attrgetter
from sklearn.cluster import DBSCAN
import hdbscan
from sklearn import preprocessing

class Utils:
    @staticmethod
    def stringified_list_of_names(people):
        return ', '.join([person.name for person in people])

    @staticmethod
    def get_taxicab_distance(location_one, location_two):
        return abs(location_one[0] - location_two[0]) + abs(location_one[1] - location_two[1])

class Car:
    def __init__(self, x, y):
        self.x_loc = x
        self.y_loc = y
        self.passengers = []
        self.pickup_requests = []

    def alter_x_loc(self, delta):
        self.x_loc = self.x_loc + delta

    def alter_y_loc(self, delta):
        self.y_loc = self.y_loc + delta

    def distance_to_movement(self, distance):
        if distance > 0:
            return 1
        elif distance <0:
            return -1
        else:
            return 0

    def move(self, destination):
        if destination == self.get_location():
            return
        if destination[0] == self.x_loc:
            distance = destination[1] - self.y_loc
            delta = self.distance_to_movement(distance)
            self.alter_y_loc(delta)
        else:
            distance = destination[0] - self.x_loc
            delta = self.distance_to_movement(distance)
            self.alter_x_loc(delta)

    def get_location(self):
        return (self.x_loc, self.y_loc)

    def show_location(self):
        print('Car located at: ' + str(self.x_loc) + "," + str(self.y_loc))

    def do_pickups(self):
        passengers_available_for_pickup = [person for person in self.pickup_requests if person.pickup == self.get_location()]
        if passengers_available_for_pickup:
            print('Passengers picked up: ' + Utils.stringified_list_of_names(passengers_available_for_pickup))
        self.passengers.extend(passengers_available_for_pickup)
        self.pickup_requests = [person for person in self.pickup_requests if person not in self.passengers]

    def do_dropoffs(self):
        passengers_at_destination = [passenger for passenger in self.passengers if passenger.dropoff == self.get_location()]
        if passengers_at_destination:
            print('Passengers dropped off: ' + Utils.stringified_list_of_names(passengers_at_destination))
        self.passengers = [passenger for passenger in self.passengers if passenger not in passengers_at_destination]

    def print_status(self):
        self.show_location()
        if self.passengers:
            print('Riding: ' + Utils.stringified_list_of_names(self.passengers))

class Passenger:
    def __init__(self, json):
        self.name = json['name']
        self.pickup = json['start']
        self.dropoff = json['end']

    def __eq__(self, other):
        return self.name == other.name and self.pickup == other.pickup and self.dropoff == other.dropoff

class Destination:
    def __init__(self, location, distance_from_car, cluster_label):
        self.location = location
        self.distance_from_car = distance_from_car
        self.cluster_label = cluster_label        

    def __eq__(self, other):
        return self.location == other.location and self.distance_from_car == other.distance_from_car
    


class Cluster:
    def __init__(self, label, destinations):
        self.destinations = destinations
        self.destination_closest_to_car = min(destinations, key=attrgetter('distance_from_car'))
        self.label = label
        self.size = len(destinations)

    def set_weight(self, normalized_size_weight, normalized_distance_weight):
        if self.label == None:
            self.weight = normalized_distance_weight
        else:
            self.weight = normalized_size_weight - normalized_distance_weight



#effing wild, but uncommenting this makes the return what I expect?
#from cluster import Cluster

class CityState:
    def __init__(self, x, y):
        self.max_rows = x
        self.max_columns = y
        self.car = Car(0,0)

    def get_locations_to_go(self):
        dropoffs = [passenger.dropoff for passenger in self.car.passengers]
        pickups = [person.pickup for person in self.car.pickup_requests]
        destinations = dropoffs + pickups
        return destinations

    def build_clusters(self, locations_to_go):
        #TODO: eps must be derived from whole grid size, not just rows
        epsilon = int(round(.10 * self.max_rows))
        clustering = DBSCAN(eps=epsilon, min_samples=2).fit(locations_to_go)
        locations_with_cluster_labels = []
        for index, location in enumerate(locations_to_go):
            if index in clustering.core_sample_indices_:
                labeled_location = (location, clustering.labels_[index])
            else:
                labeled_location = (location, None)
            locations_with_cluster_labels.append(labeled_location)
        groups = itertools.groupby(locations_with_cluster_labels, itemgetter(1))
        clusters = [Cluster(key, self.build_destinations(list(value))) for key, value in groups]

        normalized_cluster_sizes = preprocessing.scale([cluster.size for cluster in clusters])
        normalized_cluster_distances = preprocessing.scale([cluster.destination_closest_to_car.distance_from_car for cluster in clusters])
        for index, cluster in enumerate(clusters):
            cluster.set_weight(normalized_cluster_sizes[index], normalized_cluster_distances[index])
        return clusters

    def build_destinations(self, clustered_locations):
        destinations = [Destination(clustered_location[0], Utils.get_taxicab_distance(clustered_location[0], self.car.get_location()), clustered_location[1]) for clustered_location in clustered_locations]
        return destinations

    def get_next_destination(self):
        locations_to_go = self.get_locations_to_go()
        if locations_to_go:
            clusters = self.build_clusters(locations_to_go)
            selected_cluster = max(clusters, key=attrgetter('weight'))
            return selected_cluster.destination_closest_to_car
        return Destination(self.car.get_location(), 0, None)

    def increment_time(self, requestJson):
        if  requestJson:
            new_pickups = [Passenger(person) for person in requestJson]
            self.car.pickup_requests.extend(new_pickups)
        next_destination = self.get_next_destination()
        print(next_destination.location)
        self.car.move(next_destination.location)
        self.car.do_pickups()
        self.car.do_dropoffs()
        self.car.print_status()


city_state = CityState(50,50)
individual = [{'name' : 'Queequeg', 'start' : (10,10), 'end' : (40,10)}]

closer_cluster = [{'name' : 'McCavity', 'start' : (30,10), 'end' : (40,10)},
    {'name' : 'Mistoffoles', 'start' : (30,11), 'end' : (20,10)}]

farther_larger_cluster = [{'name' : 'Ishmael', 'start' : (35,4), 'end' : (4,3)},
    {'name' : 'Mieville', 'start' : (35,5), 'end' : (8,17)},
    {'name' : 'Starbuck', 'start' : (35,6), 'end' : (30,7)}]

request = individual + closer_cluster + farther_larger_cluster

#WHEN I decide which destination to choose
city_state.increment_time(request)

#THEN it is in the larger cluster even though it's further away
next_destination = city_state.get_next_destination()

print(next_destination.location)
print('FOO')

(35, 4)
Car located at: 1,0
(35, 4)
FOO


In [151]:
closest_person = [{'name' : 'Queequeg', 'start' : (25,10), 'end' : (40,10)}]
closer_cluster = [{'name' : 'McCavity', 'start' : (100,10), 'end' : (40,10)},
            {'name' : 'Mistoffoles', 'start' : (100,11), 'end' : (20,10)},
            {'name' : 'Starbuck', 'start' : (105,6), 'end' : (30,7)},
            {'name' : 'Starbuck', 'start' : (104,6), 'end' : (30,7)},
            {'name' : 'Starbuck', 'start' : (103,6), 'end' : (30,7)},
            {'name' : 'Starbuck', 'start' : (102,6), 'end' : (30,7)},
            {'name' : 'Starbuck', 'start' : (105,7), 'end' : (30,7)}]

farther_cluster = [{'name' : 'Ishmael', 'start' : (400,4), 'end' : (4,3)},
            {'name' : 'Mieville', 'start' : (400,5), 'end' : (8,17)}]
request = closest_person + closer_cluster + farther_cluster


city_state.increment_time(request)

destination = city_state.get_next_destination()
print(destination.location)

close_person = {'name' : 'McCavity', 'start' : (1,2), 'end' : (10,10)}
request = [close_person,
           {'name' : 'Ishmael', 'start' : (100,5), 'end' : (4,3)},
           {'name' : 'Mieville', 'start' : (100,6), 'end' : (8,7)}]

#WHEN I decide which destination to choose
city_state.increment_time(request)

#THEN it is in the dense cluster even though it's further away
next_destination = city_state.get_next_destination()
print(next_destination.location)


city = CityState(8,8)

print("=====NEW SCENARIO=====")
print("Should have motivation to move, 1 passenger")
example_request = [{'name' : 'Elon', 'start' : (3,5), 'end' : (8,7)}, 
                   {'name' : 'George', 'start' : (1,2), 'end' : (4,3)},
                  {'name' : 'First', 'start' : (0,0), 'end' : (1,0)}]
city.increment_time(example_request)
print("***time passes 1***")
city.increment_time([])
print("***time passes 2***")
city.increment_time([])
print("***time passes 3***")
city.increment_time([])
print("***time passes 4***")
city.increment_time([])
print("***time passes 5***")
city.increment_time([])
print("***time passes 6***")
city.increment_time([])
print("***time passes 7***")
city.increment_time([])
print("***time passes 8***")
city.increment_time([])
print("***time passes 9***")
city.increment_time([])
print("***time passes 10***")
city.increment_time([])
city.increment_time([])
print("***time passes 11***")
city.increment_time([])
print("***time passes 12***")
city.increment_time([])
print("***time passes 13***")
city.increment_time([])
city.increment_time([])
print("***time passes 14***")
city.increment_time([])
print("***time passes 15***")
city.increment_time([])
print("***time passes 16***")
city.increment_time([])

city = CityState(8,8)

print("=====NEW SCENARIO=====")
print("Should have no motivation to move, no real info")
city.increment_time([])

newcity = CityState(8,8)

print("=====NEW SCENARIO=====")
newcity.increment_time([{'name' : 'First', 'start' : (1,0), 'end' : (1,2)}])
print("***time passes 1***")
newcity.increment_time([])
print("***time passes 2***")
newcity.increment_time([])

=====NEW SCENARIO=====
Should have no motivation to move, no real info
Car located at: 0,0
=====NEW SCENARIO=====
Passengers picked up: First
Car located at: 1,0
Riding: First
***time passes 1***
Car located at: 1,1
Riding: First
***time passes 2***
Passengers dropped off: First
Car located at: 1,2
